# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [4]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [7]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [8]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [9]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [10]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55586311, 67375700)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [12]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [15]:
wv.most_similar('Texas')

[('Arkansas', 0.8065662384033203),
 ('Pennsylvania', 0.80137038230896),
 ('Florida', 0.7957906723022461),
 ('Louisiana', 0.7850519418716431),
 ('Ohio', 0.7808342576026917),
 ('Illinois', 0.7799596786499023),
 ('California', 0.7794137597084045),
 ('Maryland', 0.7784236669540405),
 ('Georgia', 0.7777647376060486),
 ('Oklahoma', 0.7741513252258301)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [16]:
wv.most_similar(negative='Texas')

[('Ex-hockey', 0.4106272757053375),
 ('PROVINCETOWN', 0.4020034372806549),
 ('Revolve', 0.3872031569480896),
 ('Optimizing', 0.3868919312953949),
 ('Headstrong', 0.382230281829834),
 ('petrifying', 0.37963807582855225),
 ('already-grim', 0.3790287673473358),
 ('Candidate\u200b', 0.36986926198005676),
 ('Late-Deciding', 0.3695816993713379),
 ('much-vaunted', 0.3626157343387604)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [17]:
wv['Texas']

array([-1.4190708 ,  0.6085702 ,  3.1900465 , -1.0088307 , -1.6699291 ,
        1.9413555 , -1.1816447 , -1.5690093 ,  0.14852424,  0.16291764,
        1.2249238 ,  0.7035637 ,  1.3737917 ,  0.2542133 , -1.1182108 ,
        0.0761895 , -0.97383344,  3.1894088 ,  0.6046889 ,  0.6575457 ,
       -2.297752  ,  2.7444232 ,  0.19326615,  0.45634496,  3.3700507 ,
        0.01296961, -0.96962106,  0.26189828,  2.805854  ,  2.0791423 ,
       -0.5321262 ,  1.0964917 ,  1.3630108 ,  1.1767721 , -1.025159  ,
        1.8916916 ,  0.35283282,  0.39619815, -0.8362726 , -1.2014602 ,
        1.7839077 ,  1.7686299 , -0.21413532, -0.12240566,  0.7630365 ,
        1.0651186 , -0.25458738,  1.903391  , -0.62646794, -0.8152379 ,
       -0.26102242, -0.73193765, -1.3228472 ,  0.24287614,  0.19655627,
       -0.16618773, -0.34242204, -2.4060595 ,  0.12604938,  3.3912761 ,
        0.6218612 , -1.3159785 ,  0.29343155, -0.49677658,  0.20445168,
       -1.0305215 , -0.44395885,  1.7804295 , -2.3919854 ,  2.40

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [20]:
wv.vectors

array([[-3.6972135e-01,  1.3256389e+00,  6.7970097e-01, ...,
        -8.3788365e-01,  1.2978485e+00, -3.6564180e-01],
       [ 9.5447809e-02, -6.1019433e-01,  4.8690090e-01, ...,
         8.2178134e-01,  2.6878347e+00,  1.2871524e+00],
       [ 1.0458847e+00, -7.4252409e-01, -1.0044367e+00, ...,
        -6.5856211e-02,  9.0250409e-01,  1.4888200e+00],
       ...,
       [-8.3623335e-02,  3.1809743e-02, -1.3520184e-02, ...,
         8.3694875e-02,  1.7253423e-02,  2.7480518e-02],
       [-2.2900065e-03,  4.2214151e-02,  2.2306576e-02, ...,
        -2.4234684e-02, -1.8690126e-02,  3.7006050e-02],
       [-6.5060295e-02, -6.2696189e-02,  2.8250888e-02, ...,
         2.8535778e-02,  9.2042296e-04,  1.8788062e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [24]:
wv.most_similar(positive=['king', 'woman'])

[('man', 0.7636635899543762),
 ('girl', 0.6741669774055481),
 ('boy', 0.6455423831939697),
 ('teenager', 0.64275062084198),
 ('musician', 0.639802098274231),
 ('guy', 0.6278486251831055),
 ('lover', 0.6182503700256348),
 ('queen', 0.6115623712539673),
 ('dog', 0.6032219529151917),
 ('bride', 0.5997775793075562)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!